<a href="https://colab.research.google.com/github/maiaufrrj/BRtrading/blob/main/pvalor_IQR_ibov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install yfinance
#!pip install networkx
#!pip install plotly
#!pip install pickle5 

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import plotly.graph_objs as go
from plotly.graph_objs import *
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from IPython.core.display import display, HTML

In [5]:
lista_ibov = pd.read_csv('https://raw.githubusercontent.com/maiaufrrj/BRtrading/main/lista_ibov.csv')['lista_ibov']

lista_acoes=[]
for i in range(len(lista_ibov)):
  lista_acoes.append(lista_ibov[i]+'.SA')
lista_ibov = list(lista_acoes)
del lista_acoes

In [6]:
acao = 'ITSA4.SA'
import pickle5 as pickle
with open('df_teste.pkl', 'rb') as f:
    data = pickle.load(f)

#df_teste = pd.read_pickle("df_teste.pkl")
df_teste = data
df = df_teste.dropna(axis='columns', thresh=200)

ModuleNotFoundError: ignored